In [ ]:
# Import libraries
import os
import warnings

import numpy as np
import pandas as pd

import gc  # Garbage collector


warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

from xgboost import XGBClassifier

In [ ]:
# load the model
import joblib
xgb_classifier = joblib.load("../input/01-starter-xgboost-implementation/xgb_classifier_v1.h5")

In [ ]:
oe = joblib.load("../input/01-starter-xgboost-implementation/oe.h5")

In [ ]:
test = pd.read_feather('../input/amexfeather/test_data.ftr')#.iloc[6000000:]

In [ ]:
# Handling date column

test["S_2_day"] = test["S_2"].dt.day
test["S_2_month"] = test["S_2"].dt.month
test["S_2_year"] = test["S_2"].dt.year

In [ ]:
drop_cols = ['D_87', 'D_88', 'D_108', 'D_111', 'D_110', 'B_39', 'D_73', 'B_42', 'D_136',
 'D_138', 'D_137', 'D_135', 'D_134', 'R_9', 'B_29', 'D_106', 'D_132', 'D_49',
 'R_26', 'D_76', 'D_66', 'D_42', 'D_142', 'D_53', 'D_82','S_2']

test.drop(columns = drop_cols,axis=1, inplace=True)

# converting pandas "categorical" dtype to numeric
cols = ["D_68", "B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126"]
test[cols] = test[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
categorical_columns = ["D_63","D_64"]
test_enc = oe.transform(test[categorical_columns])
test[categorical_columns] = test_enc

In [ ]:
_ = gc.collect()

In [ ]:
# considering only one data point per customer (latest one) as time series is not being used
test = test.groupby(['customer_ID']).nth(-1).reset_index(drop=True)

In [ ]:
# test.isna().sum()

In [ ]:
# # For numeric columns
# numeric_columns = test.select_dtypes(np.number).columns
# test[numeric_columns] = test[numeric_columns].fillna(test[numeric_columns].mean())

In [ ]:
# Score up the test dataset
test_preds = xgb_classifier.predict(test)
# test_preds.view()


### Submission

In [ ]:
# Make submission
sub_data = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')
sub_data.head()

In [ ]:
sub_data.shape

In [ ]:
sub_data['prediction'] = test_preds
sub_data.head()

In [ ]:
# Submission file
sub_data.to_csv('submission.csv', index=False)